<a href="https://colab.research.google.com/github/alyssaimani/Drug_Discovery_Acetylcholinesterase/blob/main/Drug_Discovery_acetylcholinesterase_Classification_with_ChemBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning ChemBERTa on SMILES Data for Human Acetylcholinesterase

In [32]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [33]:
import pandas as pd
df_2class = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Bioinformatics/Acetylcholinesterase/acetylcholinesterase_bioactivity_data_2class_pIC50.csv")

In [34]:
columnNames = ["canonical_smiles","bioactivity_class"]
smiles_2class = pd.DataFrame(data =df_2class, columns=columnNames)
smiles_2class

,canonical_smiles,bioactivity_class
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,active
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,active
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,inactive
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,active
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,active
...,...,...
2723,c1ccc2oc(CN(CCCCCCCNc3c4c(nc5ccccc35)CCCCC4)Cc...,inactive
2724,O=C(NCCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,active
2725,COc1cccc2cc(C(=O)NCCCCCCNc3c4c(nc5ccccc35)CCCC...,inactive
2726,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,inactive


## Integer Encoding on Bioactivity Class

In [35]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label_mapping = {"active": 1, "inactive": 0}
smiles_2class['bioactivity_code'] = le.fit_transform([label_mapping[bioactivity_class] for bioactivity_class in smiles_2class['bioactivity_class']])
smiles_2class

,canonical_smiles,bioactivity_class,bioactivity_code
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,active,1
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,active,1
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,inactive,0
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,active,1
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,active,1
...,...,...,...
2723,c1ccc2oc(CN(CCCCCCCNc3c4c(nc5ccccc35)CCCCC4)Cc...,inactive,0
2724,O=C(NCCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,active,1
2725,COc1cccc2cc(C(=O)NCCCCCCNc3c4c(nc5ccccc35)CCCC...,inactive,0
2726,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,inactive,0


## Split Dataset

In [36]:
from sklearn.model_selection import train_test_split

train_data_2class, test_data_2class, train_labels_2class, test_labels_2class = train_test_split(smiles_2class['canonical_smiles'], smiles_2class['bioactivity_code'], test_size=0.2, random_state=42)
train_data_2class, val_data_2class, train_labels_2class, val_labels_2class = train_test_split(train_data_2class, train_labels_2class, test_size=0.2, random_state=42)

In [37]:
train_2class = pd.concat([train_data_2class, train_labels_2class], axis=1)
test_2class = pd.concat([test_data_2class, test_labels_2class], axis=1)
val_2class = pd.concat([val_data_2class, val_labels_2class], axis=1)

In [38]:
print("Train Dataset: {}".format(train_2class.shape))
print("Eval Dataset: {}".format(val_2class.shape))
print("TEST Dataset: {}".format(test_2class.shape))

Train Dataset: (1745, 2)
Eval Dataset: (437, 2)
TEST Dataset: (546, 2)


## Import libraries

In [39]:
!pip install --pre deepchem
import deepchem
from rdkit import Chem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from simpletransformers.classification import ClassificationModel
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## Build Classifucation Model with ChemBERTa

In [42]:
# Check avaibility CUDA
import torch

print(torch.cuda.is_available())

True


In [43]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', args={'evaluate_each_epoch': True, 'evaluate_during_training_verbose': True, 'no_save': True, 'num_train_epochs': 10, 'auto_weights': True}) 

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE

In [44]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_396_250', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)


In [45]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## Train the Model

In [51]:
#Train the model
model.train_model(train_2class, eval_df=val_2class, output_dir='/content/PubChem_acetylcholinesterase_2class', args={'wandb_project': 'Drug Discovery - Acetylcholinesterase'})

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1745 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/219 [00:00<?, ?it/s]

(2190, 0.33510712607674403)

In [52]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_2class, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/546 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

Training loss,▆▇▅▆▅▅▃▅▄▅▃▄█▂▂▂▄▂▃▆▁▅▂▃▃▂▃▂▂▃▇▃▁▁▁▁▂▄▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr,▄▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁
Training loss,0.00167
global_step,2150
lr,0.0


In [53]:
result

{'mcc': 0.4385298236667154,
 'tp': 217,
 'tn': 176,
 'fp': 66,
 'fn': 87,
 'auroc': 0.770300946063506,
 'auprc': 0.8077251489284241,
 'acc': 0.7197802197802198,
 'eval_loss': 1.524429677837137}

In [54]:
import pickle
path = '/content/gdrive/My Drive/Colab Notebooks/Bioinformatics/Acetylcholinesterase/'

# Save the model
with open(path+'chemberta_classification.pkl', 'wb') as f:
    pickle.dump(model, f)